In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from itertools import product, permutations

import numba
from numba import njit,vectorize, jit
from tqdm import tqdm

import Estimators

In [3]:
d = 2
c = 0

In [8]:
#@njit
sig2 = 1
@njit
def GaussianKernel(x,y):
    return np.exp(-np.linalg.norm(x-y)**2/(2*sig2))

@njit
def PolynomialKernel(x,y): 
    return (x.dot(y)+c)**d
@njit
def LinearKernel(x,y): 
    return x.dot(y)

@njit
def Laplace_kernel(x, y, gamma=1):
    return 0.5 * np.exp(-gamma * np.linalg.norm(x-y))

In [9]:
def importation_mat_100(): 
    X_train_100 = []
    X_test_100 = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'_mat100.csv',delimiter= ',', header= None)
        xtrain = np.squeeze(xtrain.to_numpy())
        X_train_100.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'_mat100.csv',delimiter= ',', header= None)
        xtest = np.squeeze(xtest.to_numpy())
        X_test_100.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train_100,X_test_100,Y_train

def importation(): 
    X_train = []
    X_test = []
    Y_train = []
    for i in range(3): 
        xtrain = pd.read_csv('data/Xtr'+str(i)+'.csv',delimiter= ',', header= None)
        xtrain = xtrain.iloc[1:,1].to_numpy()
        X_train.append(xtrain)
    
        xtest = pd.read_csv('data/Xte'+str(i)+'.csv',delimiter= ',', header= None)
        xtest = xtest.iloc[1:,1].to_numpy()
        X_test.append(xtest)
    
        Y_train.append(pd.read_csv('data/Ytr'+str(i)+'.csv',delimiter= ',')['Bound'].to_numpy())
    return X_train,X_test,Y_train

X_train,X_test,Y_train = importation()
X_train_100, X_test_100,Y_train = importation_mat_100()

In [10]:
X_traintest = []
for i in range(3): 
    X_traintest.append(np.concatenate((X_train[i],X_test[i])))


In [11]:
def to_Kernel_train(X, Kernel): 
    length = X.shape[0]
    mat_K = np.zeros((length,length))
    for i in range(length):
        x_i = np.squeeze(X[i])
        for j in range(i,length): 
            x_j = np.squeeze(X[j])
            value = Kernel(x_i,x_j)
            mat_K[i,j] = value
            mat_K[j,i] = value 
    return mat_K

#@njit 
def to_Kernel_test(Xtrain,Xtest,Kernel):
    '''
    takes the training data input Xtrain and the test data Xtest and computes the Kernel_test. 
    
    The length of the resulting Kernel_test will be (nb_traing_samples, nb_testing_samples) 
    '''
    length_train = Xtrain.shape[0]
    length_test = Xtest.shape[0]
    bimat_K = np.zeros((length_train,length_test))
    for i in range(length_train):
        x_i = np.squeeze(Xtrain[i])
        for j in range(length_test): 
            x_j = np.squeeze(Xtest[j])
            bimat_K[i,j] = Kernel(x_i,x_j)
    return bimat_K

def standardize(K): 
    '''
    standardize the given matrix K
    '''
    U = np.full(K.shape,1/K.shape[0])
    I = np.eye(K.shape[0])
    return (I-U)@K@(I-U)

In [13]:
# Compute phi for Spectrum kernel
def phi_spectrum(x,k,U):
    """U: list of the sequences of size k to look for in x"""
    phi_spec = np.zeros(len(U))
    for j, u in enumerate(U):
        for i in range(len(x)-k+1):
            if x[i:i+k] == u:
                phi_spec[j] +=1
    return phi_spec

#Compute phi for Substring kernel
def l(i):
    return i[-1] - i[0] + 1

def I(k,n):
    I = set()
    for ele in permutations(range(0,n),k):
        I.add(tuple(sorted(list(ele))))
    return list(I)

def phi_substring(x,k,U,lamb=0.5): #fonctionne (testé avec l'exemple page 392)
    """U: list of the sequences of size k to look for in x"""
    phi = np.zeros(len(U))
    I_kn = I(k,len(x))
    for j, u in enumerate(U):
        for i in I_kn:
            x_i = "".join([x[idx]  for idx in i])
            if x_i==u:
                phi[j] += lamb**l(i)
    return phi


def make_dict_phi(X, phi,k):
    U = [''.join(letter) for letter in product('ACGT', repeat=k)]
    phi_dict = {seq:phi(seq,k,U) for seq in tqdm(X)}
    return phi_dict


In [14]:
##CONSTRUCTION DES DICTIONNAIRES. On fait ça pour eviter de les recalculer à chaque fois
dict_spectrum_traintest = [0]*10

#dict_spectrum_traintest[3] = make_dict_phi(X_traintest[0],phi_spectrum,3)
#dict_spectrum_traintest[4] = make_dict_phi(X_traintest[0],phi_spectrum,4)

#dict_spectrum_traintest[5] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,5)
#dict_spectrum_traintest[6] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,6)
dict_spectrum_traintest[7] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,7)
#dict_spectrum_traintest[8] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,8)
#dict_sub_traintest_2 = make_dict_phi(X_traintest[0], phi_substring,2)

#dict_sub_traintest_3 = make_dict_phi(X_traintest[0], phi_substring,3)

100%|██████████| 9000/9000 [06:13<00:00, 24.08it/s]


In [15]:
dict_spectrum_traintest[7] = make_dict_phi(np.concatenate(X_traintest),phi_spectrum,7)

100%|██████████| 9000/9000 [24:52<00:00,  6.03it/s]


In [16]:
def make_K_spectrum(k): 
    '''
    template function to build a K_spectrum_function. 
    We need to have one function for each kernel functionin order to compute efficiently the Kernel matrix 
    '''
    def K_spectrum(x,y): 
        value = np.sum([dict_spectrum_traintest[k][str(x)] * dict_spectrum_traintest[k][str(y)]])
        return value 
    return K_spectrum
K_spectrum_5 = make_K_spectrum(5)
K_spectrum_6 = make_K_spectrum(6)
K_spectrum_7 = make_K_spectrum(7)

In [85]:
dict_spectrum_traintest[5][X_train[0][1]]

array([0., 0., 0., ..., 0., 2., 3.])

In [17]:
def get_first_Kernels(Kernel): 
    '''
    gets the first Kernel_train and Kernel_test. This is to 
    Permet d'avoir les Kernels de train et de test, au moins pour tester le modèle, voir ce qu'il donne sur le premier dataset. 
    '''
    K_train = to_Kernel_train(X_train[0], Kernel)
    K_test = to_Kernel_test(X_train[0], X_test[0], Kernel)
    return K_train,K_test 

def get_list_Kernels(Kernel) : 
    '''
    Permet d'avoir la liste des Kernel_train et Kernel_test pour chaque deteset. renvoie deux listes : la premiere contient 
    les 3 premiers Kernel_train des 3 datasets. Le deuxieme renvoie les Kernel_test des 3 datasets. 
    
    Ne marche qu'avec les kernels qui prenent des strings en entree. 
    '''
    list_K_train = list()
    list_K_test = list()
    for i in tqdm(range(3)): 
        list_K_train.append(to_Kernel_train(X_train[i], Kernel))
        list_K_test.append(to_Kernel_test(X_train[i],X_test[i], Kernel))
    return list_K_train,list_K_test

def get_list_Kernels(Kernel) : 
    '''
    Permet d'avoir la liste des Kernel_train et Kernel_test pour chaque deteset. renvoie deux listes : la premiere contient 
    les 3 premiers Kernel_train des 3 datasets. Le deuxieme renvoie les Kernel_test des 3 datasets. 
    
    Ne marche qu'avec les kernels qui prenent des strings en entree. 
    '''
    list_K_train = list()
    list_K_test = list()
    for i in tqdm(range(3)): 
        list_K_train.append(to_Kernel_train(X_train[i], Kernel))
        list_K_test.append(to_Kernel_test(X_train[i],X_test[i], Kernel))
    return list_K_train,list_K_test

In [93]:
list_K_train_spectrum_6, list_K_test_spectrum_6 = get_list_Kernels(K_spectrum_6)  

100%|██████████| 3/3 [05:33<00:00, 111.33s/it]


In [95]:
list_K_train_spectrum_5, list_K_test_spectrum_5 = get_list_Kernels(K_spectrum_5)  

100%|██████████| 3/3 [05:21<00:00, 107.27s/it]


In [18]:
list_K_train_spectrum_7, list_K_test_spectrum_7 = get_list_Kernels(K_spectrum_7)  

100%|██████████| 3/3 [09:34<00:00, 191.66s/it]


In [37]:
def grid_search_cv(model, Kernel_train, Y_train, parameters): 
    scores = list()
    for parameter in parameters : 
        model.set_parameter(parameter)
        scores.append(model.cross_val(Kernel_train,Y_train, 5))
        print('We have tested  : ', parameter)
    arg_max = np.argmax(np.array(scores))
    print('###########BEST_PARAM = ',parameters[arg_max])
    return parameters[arg_max]
KRR_spectrum_7 = Estimators.KRR(Kernel = K_spectrum_7)
parameters = np.linspace(0.0001,0.2,30)
#parameters = 10**np.linspace(-10,0,15)
grid_search_cv(KRR_spectrum_7, list_K_train_spectrum_7[2], Y_train[2], parameters)

5it [00:00, 14.09it/s]
0it [00:00, ?it/s]

Score :  0.7025
We have tested  :  0.0001


5it [00:00, 10.87it/s]


Score :  0.72
We have tested  :  0.006993103448275863


5it [00:00,  9.81it/s]
0it [00:00, ?it/s]

...

5it [00:00, 11.11it/s]
0it [00:00, ?it/s]

Score :  0.735
We have tested  :  0.19310689655172414


5it [00:00, 13.13it/s]

Score :  0.743
We have tested  :  0.2
###########BEST_PARAM =  0.06903103448275863


0.06903103448275863

In [38]:
def download_results(model,list_Kernel_train,list_Kernel_test, parameters, name_dossier):
    Y_predicted = []
    for i in range(3): 
        model.set_parameter(parameters[i])
        model.fit(list_Kernel_train[i], Y_train[i])
        Y_predicted.append(model.predict(list_Kernel_test[i])*1)
        print('Model {} Predicted'.format(i))

    d = { 'Id' : np.arange(3000), 'Bound' : np.concatenate(Y_predicted)}
    out = pd.DataFrame(data=d)
    out.to_csv('predictions_KM'+name_dossier+'.csv', index=False)

In [40]:
best_parameters = [0.2057,1.1275,0.06903]
download_results(KRR_spectrum_7,list_K_train_spectrum_7,list_K_test_spectrum_7,best_parameters,'K_spectrum_7')

Model 0 Predicted
Model 1 Predicted
Model 2 Predicted


In [86]:
def downloads_from_voting(models, lists_Kernel_train,lists_Kernel_test,parameterss, name_dossier): 
    '''
    Takes some models and computes the results by a voting process. 
    lists_Kernel_train is a list of list_Kernel_train. same for lists_Kernel_test. 
    parameters is a list of list of parameter (the best parameter possible)
    name_dossier is the name of the dossier you want. should be informative
    It automatically downloads the predictions
    '''
    Y_predicted = []
    length = len(models)
    for i in range(3): 
        y_pred = list()
        for model,list_Kernel_train,list_Kernel_test,parameters in zip(models, lists_Kernel_train, lists_Kernel_test, parameterss): 
            model.set_parameter(parameters[i])
            model.fit(list_Kernel_train[i], Y_train[i])
            y_pred.append(model.predict(list_Kernel_test[i]))
        y_pred = np.array(y_pred)
        y_pred[:,0][0]= False
        y_pred[0] = y_pred[0]*2-0.5
        print(y_pred[:,0])
        Y_predicted.append(np.sum(y_pred,axis = 1)>0.5)
        
    ##### rajouter la fin et verifier que ca marche, plus mettre set_parameter dans le SVM de Louis 
            

In [87]:
models = [KRR_spectrum_7,KRR_spectrum_7]
lists_Kernel_train = [list_K_train_spectrum_7, list_K_train_spectrum_7]
lists_Kernel_test = [list_K_test_spectrum_7, list_K_test_spectrum_7]
parameterss = [[0.2057,1.1275,0.06903], [0.3636,0.21428,0.2057]]
name_dossier = 'voting_two_K_spectrum_7'
downloads_from_voting(models, lists_Kernel_train, lists_Kernel_test, parameterss,name_dossier)

[ True  True]
[ True  True]
[ True False]
